In [1]:
import sys
import os

os.chdir('code')
from cfn_model_generation import * 
os.chdir('..')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import trange


In [13]:
n_internal_verts = 50 #Number of internal vertices in the tree
batch_size = 1000 #Number of spins per batch.
num_batches = 50 #Number of batches per tree
tree_types = [2, 3, 4, 5,6,7,8] #Degree types for d-regular tree

trials_per_tree = batch_size * num_batches # Trials per tree
total_trials = trials_per_tree * num_trees # Total number of trials


delta = 1/10  #Value of delta. Theoretical max is delta = 1/924
C_val = .5 #C_{2.1} and C_{2.2}
c_val = 0.25 #c_{2.1} and c_{2.2}

C_upperTail = 4/5 * (9 * C_val**2 / (2*c_val) + 2* C_val)**2 #C_{2.7}
c_lowerTail = 1 - 2*c_val/(3*C_val) #c_{2.8}

probability_upper_tail_constant = 7*C_val #c_{2.7}
probability_lower_tail_constant = 78*C_val**2 #C_{2.8}

upper_tail_sample_threshold = 1 - C_upperTail* delta**2
lower_tail_sample_threshold = -c_lowerTail

upper_tail_probability = 1 - probability_upper_tail_constant*delta
lower_tail_probability = probability_lower_tail_constant * delta**2

print(f"The theoretical upper-tail bound is {upper_tail_probability:.4} and lower-tail bound is {lower_tail_probability:.4} for delta = {delta}.")

num_bins = max(500,int(10/delta))
bins = np.linspace(-1,1, num_bins+1)




The theoretical upper-tail bound is 0.65 and lower-tail bound is 0.195 for delta = 0.1.


In [14]:
histograms = [np.histogram([], bins)[0] for _ in tree_types]
empircal_count_above_threshold = []
empircal_count_below_threshold = []

for ind, deg in enumerate(tree_types):
    print(f'Working on tree {ind+1}/{len(tree_types)}.')
    empircal_count_above_threshold.append(0)
    empircal_count_below_threshold.append(0)
    Z = histograms[ind]
    deg_sequence = [deg for deg in range(n_internal_verts)]
    root = GeneralizedFoataFuchs(deg_sequence, delta, C_val = C_val, c_val = c_val)
    for i in trange(num_batches):
        attempt = root.constructMagnetization(batch_size)
        root.magnetizations = None

        empircal_count_above_threshold[-1] += sum(attempt >= upper_tail_sample_threshold)
        empircal_count_below_threshold[-1] += sum(attempt <= lower_tail_sample_threshold)

        binned_attempt = np.histogram(attempt, bins)[0]

        Z = Z+binned_attempt
    histograms[ind] = Z
    p_up = empircal_count_above_threshold[-1]/trials_per_tree
    p_down = empircal_count_below_threshold[-1]/trials_per_tree
    print(f"Proportion of samples above upper threshold {p_up}.")
    print(f"Proportion of samples above upper threshold {p_down}.")


Working on tree 1/7.


100%|██████████| 50/50 [00:32<00:00,  1.54it/s]


Proportion of samples above upper threshold [1.].
Proportion of samples above upper threshold [0.].
Working on tree 2/7.


100%|██████████| 50/50 [00:30<00:00,  1.65it/s]


Proportion of samples above upper threshold [1.].
Proportion of samples above upper threshold [0.].
Working on tree 3/7.


...

Proportion of samples above upper threshold [1.].
Proportion of samples above upper threshold [0.].
Working on tree 6/7.


100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


Proportion of samples above upper threshold [1.].
Proportion of samples above upper threshold [0.].
Working on tree 7/7.


 20%|██        | 10/50 [00:06<00:26,  1.49it/s]


KeyboardInterrupt: 

In [17]:
histograms[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,